### Network growth

1. Search "History" in section headings.
2. Find a list of years.
3. Use the first one as when the birth year of the idea.
4. If no "History" section, then look backwards in the graph to a page that does, and use that.

#### Visualization
* Use ~~[bokeh](https://bokeh.pydata.org/en/latest/docs/user_guide/graph.html)~~ ~~[webweb](https://webwebpage.github.io/)~~ [d3](https://observablehq.com/@d3/force-directed-graph).
* Use slider to show growth in network

### Label with years

In [1]:
import networkx as nx

path_saved = '/Users/harangju/Box Sync/Research/my papers/wikipedia paper/data/graphs/undated/'
# natural & physical sciences
topics = ['anatomy', 'biochemistry', 'cognitive science', 'evolutionary biology',
          'genetics', 'immunology', 'molecular biology']
topics += ['chemistry', 'biophysics', 'energy', 'optics', 
           'earth science', 'geology', 'meteorology']
# philosophy
# topics += []
topics += ['philosophy of language', 'philosophy of law', 
           'philosophy of mind', 'philosophy of science']
# social sciences
topics += ['economics', 'accounting', 'education', 'linguistics', 'law', 'psychology', 'sociology']
# technology & applied sciences
topics += ['electronics', 'software engineering', 'robotics']
topics += ['physics']

graphs = {}
for topic in topics:
    graphs[topic] = nx.read_gexf(path_saved + topic + '.gexf')
graphs

{'anatomy': <networkx.classes.digraph.DiGraph at 0x7fdab0b6f5f8>,
 'biochemistry': <networkx.classes.digraph.DiGraph at 0x7fda804cee80>,
 'cognitive science': <networkx.classes.digraph.DiGraph at 0x7fda80574668>,
 'evolutionary biology': <networkx.classes.digraph.DiGraph at 0x7fda804a5710>,
 'genetics': <networkx.classes.digraph.DiGraph at 0x7fda808647b8>,
 'immunology': <networkx.classes.digraph.DiGraph at 0x7fda806bbf28>,
 'molecular biology': <networkx.classes.digraph.DiGraph at 0x7fda8060d080>,
 'chemistry': <networkx.classes.digraph.DiGraph at 0x7fda80903b38>,
 'biophysics': <networkx.classes.digraph.DiGraph at 0x7fda80c42668>,
 'energy': <networkx.classes.digraph.DiGraph at 0x7fda80930f98>,
 'optics': <networkx.classes.digraph.DiGraph at 0x7fda80877a58>,
 'earth science': <networkx.classes.digraph.DiGraph at 0x7fda8099cb38>,
 'geology': <networkx.classes.digraph.DiGraph at 0x7fda80985438>,
 'meteorology': <networkx.classes.digraph.DiGraph at 0x7fda806bb2b0>,
 'philosophy of langu

In [2]:
import wiki

path_base = '/Users/harangju/Developer/data/wiki/'
name_xml = 'enwiki-20190801-pages-articles-multistream.xml.bz2'
name_index = 'enwiki-20190801-pages-articles-multistream-index.txt.bz2'
path_xml = path_base + name_xml
path_index = path_base + name_index
dump = wiki.Dump(path_xml, path_index)

In [5]:
for topic, graph in graphs.items():
    print('Topic: ' + topic)
    for node in graph.nodes:
        dump.load_page(node)
        graph.nodes[node]['year'] = dump.years[0] if len(dump.years)>0 else []

Topic: anatomy
Topic: biochemistry
Topic: cognitive science
Topic: evolutionary biology
Topic: genetics
Topic: immunology
Topic: molecular biology
Topic: chemistry
Topic: biophysics
Topic: energy
Topic: optics
Topic: earth science
Topic: geology
Topic: meteorology
Topic: philosophy of language
Topic: philosophy of law
Topic: philosophy of mind
Topic: philosophy of science
Topic: economics
Topic: accounting
Topic: education
Topic: linguistics
Topic: law
Topic: psychology
Topic: sociology
Topic: electronics
Topic: software engineering
Topic: robotics
Topic: physics


In [8]:
path_dated_unfilled = '/Users/harangju/Box Sync/Research/my papers/wikipedia paper/data/graphs/dated/unfilled/'
for topic, graph in graphs.items():
    nx.write_gexf(graph, path_dated_unfilled + topic + '.gexf')

In [9]:
graphs['robotics'].nodes['Robotics']

{'label': 'Robotics', 'year': 1900}

#### Fill in nodes without years

In [10]:
def fill_empty_nodes(graph, full_parents=True):
    """
    Returns
    -------
    bool
        whether at least 1 empty node was filled
    """
    empty_nodes = [n for n in graph.nodes if not graph.nodes[n]['year']]
    for node in empty_nodes:
        years = [graph.nodes[p]['year'] for p in graph.predecessors(node)]
        if not years:
            continue
        if full_parents:
            if [] not in years:
                graph.nodes[node]['year'] = max(years)
                return True
        else:
            years_filtered = [y for y in years if y]
            if years_filtered:
                graph.nodes[node]['year'] = max(years_filtered)
                return True
    return False

In [12]:
for topic, graph in graphs.items():
    print('Topic:', topic)
    nodes = graph.nodes
    print('\tFilling in nodes with full parents...')
    nodes_filled = True
    while nodes_filled:
        nodes_filled = fill_empty_nodes(graph, full_parents=True)
    print('\tFilling in nodes without full parents...')
    nodes_filled = True
    while nodes_filled:
        nodes_filled = fill_empty_nodes(graph, full_parents=False)

Topic: anatomy
	Filling in nodes with full parents...
	Filling in nodes without full parents...
Topic: biochemistry
	Filling in nodes with full parents...
	Filling in nodes without full parents...
Topic: cognitive science
	Filling in nodes with full parents...
	Filling in nodes without full parents...
Topic: evolutionary biology
	Filling in nodes with full parents...
	Filling in nodes without full parents...
Topic: genetics
	Filling in nodes with full parents...
	Filling in nodes without full parents...
Topic: immunology
	Filling in nodes with full parents...
	Filling in nodes without full parents...
Topic: molecular biology
	Filling in nodes with full parents...
	Filling in nodes without full parents...
Topic: chemistry
	Filling in nodes with full parents...
	Filling in nodes without full parents...
Topic: biophysics
	Filling in nodes with full parents...
	Filling in nodes without full parents...
Topic: energy
	Filling in nodes with full parents...
	Filling in nodes without full paren

In [13]:
path_dated_filled = '/Users/harangju/Box Sync/Research/my papers/wikipedia paper/data/graphs/dated/filled/'
for topic, graph in graphs.items():
    nx.write_gexf(graph, path_dated_filled + topic + '.gexf')

#### Load graphs

In [ ]:
topics = ['biochemistry']
path_base = '/Users/harangju/Box Sync/Research/my papers/wikipedia paper/data/graphs/dated/filled/'

In [ ]:
import networkx as nx

graphs = {}
for topic in topics:
    graphs[topic] = nx.read_gexf(path_base + topic + '.gexf')
graphs

### Visualize growth

* gephi [tutorial](https://seinecle.github.io/gephi-tutorials/generated-html/converting-a-network-with-dates-into-dynamic.html)

### Clique community growth

In [ ]:
import networkx as nx

In [ ]:
# cliques = nx.enumerate_all_cliques(nx.Graph(graphs['biochemistry']))
cliques = nx.find_cliques(nx.Graph(graphs['biochemistry']))
list(cliques)[100:110]

### Persistent homology